In [47]:
import torch
import torchvision
from torch import nn
from PIL import Image
import numpy as np
from torchvision import transforms
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from torch import optim


In [2]:
cp /content/drive/MyDrive/leadersd/images.tar images.tar

In [ ]:
!tar -xvf images.tar

In [6]:
import os 
import shutil

In [8]:
print(len(os.listdir('Images/')))

120


In [77]:
def load_dataset(data_path):
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Resize((100, 100)),
                        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225]) 

        ])
    )
    return train_dataset

def get_loader(train_dataset):
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=64,
        num_workers=0,
        shuffle=True
    )
    return train_loader

In [160]:
dataset = load_dataset('Images')
print(len(dataset))

10400


In [215]:
model = torchvision.models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [216]:
for layer in model.parameters():
  layer.required_grad = True

In [222]:
n_inputs = 1000
n_classes = 61
add_layers = nn.Sequential(
                      nn.Linear(n_inputs, 512),
                      nn.Tanh(),
                      nn.Dropout(0.3),
                      nn.Linear(512, 256),
                      nn.ReLU(),
                      nn.Linear(256, n_classes)
                      )

In [223]:
torch.manual_seed(42)
train_data, val_data = torch.utils.data.random_split(dataset, (8500, 10400-8500))
train_loader = get_loader(train_data)
val_loader = get_loader(val_data)

In [224]:
print(len(train_loader))

133


In [225]:
def acc(a, b):
  return sum(a==b).item()/a.shape[0]

In [226]:
torch.manual_seed(42)
model = model.to('cuda')
add_layers.to('cuda')
model.eval()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(add_layers.parameters())
for epochs in range(10):
  add_layers.train()
  for (data, target) in tqdm(train_loader):
    data = data.to('cuda')
    target = target.to('cuda')
    optimizer.zero_grad()
    predict = add_layers(model(data))
    loss = criterion(predict, target)
    loss.backward()
    optimizer.step()
  accuracy = []
  add_layers.eval()
  for (data, target) in tqdm(val_loader):
    data = data.to('cuda')
    target = target.to('cuda')
    predict = model(data)
    predict = add_layers(predict)
    predict = predict.to('cpu').detach().numpy()
    ans = 0
    for (arr, targ) in zip(predict, target.to('cpu').detach().numpy()):
      ind = arr.argsort()[-3:][::-1]
      ans += sum(ind == targ)
    accuracy.append(ans / 64)
  print(f'epoch = {epochs + 1}: {np.mean(accuracy)}')

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

epoch = 1: 0.753125


  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

epoch = 9: 0.7880208333333333


  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

epoch = 10: 0.7807291666666667


In [227]:
accuracy = []
add_layers.eval()
for (data, target) in tqdm(val_loader):
    data = data.to('cuda')
    target = target.to('cuda')
    predict = model(data)
    predict = add_layers(predict)
    predict = predict.to('cpu').detach().numpy()
    ans = 0
    for (arr, targ) in zip(predict, target.to('cpu').detach().numpy()):
      ind = arr.argsort()[-3:][::-1]
      ans += sum(ind == targ)
    accuracy.append(ans/64)
print(f'epoch = {epochs + 1}: {np.mean(accuracy)}')

  0%|          | 0/30 [00:00<?, ?it/s]

epoch = 10: 0.7807291666666667


In [159]:
print(os.listdir('Images'))

['n02099601-golden_retriever', 'n02110063-malamute', 'n02097209-standard_schnauzer', 'n02089867-Walker_hound', 'n02108089-boxer', 'n02101006-Gordon_setter', 'n02102318-cocker_spaniel', 'n02089078-black-and-tan_coonhound', 'n02099712-Labrador_retriever', 'n02102177-Welsh_springer_spaniel', 'n02102480-Sussex_spaniel', 'n02095570-Lakeland_terrier', 'n02113799-standard_poodle', 'n02090379-redbone', 'n02100877-Irish_setter', 'n02092002-Scottish_deerhound', 'n02087394-Rhodesian_ridgeback', 'n02111889-Samoyed', 'n02088632-bluetick', 'n02105251-briard', 'n02111129-Leonberg', 'n02115641-dingo', 'n02096051-Airedale', 'n02088238-basset', 'n02099267-flat-coated_retriever', 'n02102973-Irish_water_spaniel', 'n02093754-Border_terrier', 'n02110185-Siberian_husky', 'n02111500-Great_Pyrenees', 'n02111277-Newfoundland', 'n02106550-Rottweiler', 'n02107142-Doberman', 'n02106030-collie', 'n02108551-Tibetan_mastiff', 'n02101388-Brittany_spaniel', 'n02092339-Weimaraner', 'n02093256-Staffordshire_bullterrier',

In [175]:
a = np.array([1, 2, 3, 4 , 5])
print(a.argsort()[-3:][::-1])

[4 3 2]


In [228]:
torch.save(add_layers.state_dict(),
           '/content/drive/MyDrive/leadersd/model_classification_dog_breeds.pt')